In [1]:
import sys
import os
import copy
import numpy as np
import scipy as sci
sys.path.append("../../TychePlot/")
from Plot_processing import calc, calc_plot_list, export_data, add_to_plot, plot
from SpectraPlot import SpectraPlot
from Data import Data
from multiprocessing import Pool
import dill # dill installed with pip install dill (pickling functions fails otherwise)
#multithreaded=True #pickler fails in pool mode so better singlethreaded

In [2]:
#columns
wavelength=1
intensity_wavelength=2
spectralRadiance_wavelength=3
energy=4
intensity_energy=5
spectralRadiance_energy=6

In [3]:
#Parameters

## rte
multithreaded=True

## files
input_file_path="spectra/"
output_file_path="spectra_plot/"
corr_output_file_path=""
input_file_ending=".csv"
fill="_"
name="XXXI"
subdir=""

## figure
figscale=0.6
present_width_pt=665.79451
present_HW_ratio=0.82

## plot data
labelList=[
            r"CsPbBrI\textsubscript{2}-EtAc-washed",
            r"Ir(ppy)\textsubscript{3}", 
            r"CsPbBr\textsubscript{2}Cl",
            r"CsPbBr\textsubscript{2}Cl-PEIE",
]

present={
            #"customFontsize":[8,8,4,6,6],
            "fig_width_pt":present_width_pt*figscale,
            "scaleX":(present_width_pt*figscale)/SpectraPlot.fig_width_default_pt,
            "fixedFigWidth":True,
            "HWratio":present_HW_ratio,
            "titleBool":False,
            "filenamePrefix":output_file_path+"presentation/"
}

In [4]:
files = os.listdir(input_file_path)
files = [file[:-4] for file in files if input_file_ending in file]
files.sort()
#files=files[:12]+files[-2:]
#files=[files[2],files[4]]
files

['XXXI']

In [5]:
desiredPlots=[
                {
                    "xCol":wavelength,
                    "xCol2":energy,
                    "yCol":intensity_wavelength,
                    "custom":
                    {
                        "rainbowMode":True
                    }
                },
                {
                    "xCol":wavelength,
                    "xCol2":energy,
                    "yCol":intensity_wavelength,
                    "custom":
                    {
                        "rainbowMode":False
                    }
                },
                {
                    "xCol2":wavelength,
                    "xCol":energy,
                    "yCol":intensity_energy,                    
                    
                }
]

In [6]:
fileList=[[input_file_path+file+input_file_ending] for file in files]
optionalParameters={
                    "customLims":False, # better pick False running first Time
                    "xOrigLims":[# Limits prior to calc in volt
                                    None,#None
                                    None,#wavelength
                                    None,#spectralRadiance
                                ],
                    "yAxisLims":[# Limits for the Plot
                                    None,#None
                                    None,#wavelength
                                    None,#spectralRadiance
                                  ],
                    "scaled":False
}
inputParameters={
                        "titleBool":False,
                        "customFontsize":[12,12,6,10,10],
                        "filenamePrefix":output_file_path+subdir,
                        #"bgYCol":[2],
                        "fitColors":['#000000','#d62728','#2ca02c','#9467bd','#8c564b','#e377c2','#7f7f7f','#ff7f0e','#bcbd22','#17becf','#f8e520'],
                        #"fileFormat":{"separator":",", "skiplines":75},
        
}
inputParameters_scaled={
                        "titleBool":False,
                        "customFontsize":[6,6,4,8,8],
                        "filenamePrefix":output_file_path+subdir,
                        #"bgYCol":[2],
                        "fitColors":['#000000','#d62728','#2ca02c','#9467bd','#8c564b','#e377c2','#7f7f7f','#ff7f0e','#bcbd22','#17becf','#f8e520'],
                        "scaleX":0.335,
                        #"fileFormat":{"separator":",", "skiplines":75},
        
}

In [7]:
groups=[
        ((0,0,2),),
        ((0,1,3),),
        ((0,2,5),),
        ((0,3,7),),
]
group_names=[
            "iodine",
            "irppy",
            "clorine",
            "clorine-PEIE",
]
group_specific_input=[
                        {
                            "desiredPlots":desiredPlots,
                        },
                        {
                            "desiredPlots":desiredPlots,
                        },
]*2

In [8]:
plot_prop_list=[]
feature="default"
for group,group_name,specific_input in zip(groups,group_names,group_specific_input):
    fileList_loc=[]
    spectra_loc=[]
    labels_loc=[]
    validYCol_loc=[]
    try:
        inputParameters_loc=copy.deepcopy(specific_input["inputParameters"])
    except:
        inputParameters_loc=copy.deepcopy(inputParameters)
    try:
        optionalParameters_loc=copy.deepcopy(specific_input["optionalParameters"])
    except:
        optionalParameters_loc=copy.deepcopy(optionalParameters)
    try:
        desiredPlots_loc=copy.deepcopy(specific_input["desiredPlots"])
    except:
        desiredPlots_loc=copy.deepcopy(desiredPlots)
    for pos in group:
        if isinstance(pos, tuple):
            fileList_loc.append(fileList[pos[0]])
            labels_loc.append(labelList[pos[1]])
            validYCol_loc.append([pos[2]])
        else:
            fileList_loc.append(fileList[pos])
            labels_loc.append(labelList[pos])
    inputParameters_loc["labels"]=labels_loc
    inputParameters_loc["validYTable"]=validYCol_loc
    plot_prop_list.append(
        {
             "name":name+fill+group_name,
             "fileList":fileList_loc,
             "desiredPlots":desiredPlots_loc,
             "present_params":present,
             "inputParameters":inputParameters_loc,
             "plot_class":SpectraPlot,
             "optionalParameters":optionalParameters_loc,
             "feature":feature,
             "multithreaded":multithreaded
        }
        )
result = calc_plot_list(plot_prop_list, multithreaded=multithreaded)

In [9]:
def update_plot(output, 
                     files=None, 
                     corr_output_file_path=corr_output_file_path,
                     list_of_fitList=None,
                     doNotFit=False,
                     legLoc=0,
                     desiredPlotIndex=0,
                    ):
    plots=[output[n][desiredPlotIndex][0] for n in range(0,len(output))]
    if list_of_fitList is not None and len(list_of_fitList)==len(plots):
        for plot,fitList in zip(plots, list_of_fitList):
            plot.fitList=fitList
            plot.doNotFit=doNotFit
            plot.legLoc=legLoc
    for plot in plots:
        if plot.filenamePrefix[-len(corr_output_file_path):] != corr_output_file_path:
            plot.filenamePrefix+=corr_output_file_path
    result=calc_plot_list(plots, multithreaded=multithreaded, directPlotInput=True)
    return result

In [10]:
spectra_for_fit=result[0:4]
spectra_for_fit

[[[<SpectraPlot.SpectraPlot at 0x7f03d27cb100>,
   'spectra_plot/XXXI_iodine_spectra_NormalisedIntensity_rainbow.pdf'],
   'spectra_plot/XXXI_iodine_spectra_NormalisedIntensity.pdf'],
   'spectra_plot/XXXI_iodine_spectra_NormalisedIntensityvsEnergy.pdf']],
 [[<SpectraPlot.SpectraPlot at 0x7f03d2540790>,
   'spectra_plot/XXXI_irppy_spectra_NormalisedIntensity_rainbow.pdf'],
   'spectra_plot/XXXI_irppy_spectra_NormalisedIntensity.pdf'],
   'spectra_plot/XXXI_irppy_spectra_NormalisedIntensityvsEnergy.pdf']],
 [[<SpectraPlot.SpectraPlot at 0x7f03d2540760>,
   'spectra_plot/XXXI_clorine_spectra_NormalisedIntensity_rainbow.pdf'],
   'spectra_plot/XXXI_clorine_spectra_NormalisedIntensity.pdf'],
   'spectra_plot/XXXI_clorine_spectra_NormalisedIntensityvsEnergy.pdf']],
 [[<SpectraPlot.SpectraPlot at 0x7f04119d2b80>,
   'spectra_plot/XXXI_clorine-PEIE_spectra_NormalisedIntensity_rainbow.pdf'],
   'spectra_plot/XXXI_clorine-PEIE_spectra_NormalisedIntensity.pdf'],
   'spectra_plot/XXXI_clorine-PEI

In [11]:
list_of_fitList=[
                    [
                        [#iodine
                             ([2.0,1.65],[2.1,1.6],SpectraPlot.doubleGauss,(1.8,0.1,0.07,1.8,0.01,0.03),(3.0,0.4),None,{"maxfev":100000}),
                        ]
                    ],
                    [
                        [#irppy
                             ([2.55,1.7],[2.6,1.6],SpectraPlot.tripleGauss,(2.4,0.15,0.07,2.3,0.08,0.05,2.2,0.03,0.05),(3.0,0.4),None,{"maxfev":100000}),
                        ]
                    ],
                    [
                        [#chlorine
                             ([2.85,2.6],[3.1,2.3],SpectraPlot.tripleGauss,(2.8,0.15,0.07,2.73,0.08,0.05,2.65,0.07,0.05),(1.8,0.4),None,{"maxfev":100000}),
                        ]
                    ],
                    [
                        [#chlorine-PEIE
                             ([2.85,2.6],[3.1,2.3],SpectraPlot.tripleGauss,(2.8,0.15,0.07,2.73,0.08,0.05,2.65,0.07,0.05),(1.8,0.4),None,{"maxfev":100000}),
                        ]
                    ],
]

In [12]:
spectra_fit = update_plot(copy.copy(spectra_for_fit), corr_output_file_path=corr_output_file_path+"fit/", list_of_fitList=list_of_fitList, doNotFit=False, desiredPlotIndex=2)